# Challenge 21: Deep Learning
## deep_learning_challenge.ipynb
The goal is to build a model to predict success of projects funded by the Alphabet Soup Charity.
The data set describes hostorical proposal and their outcome and has these features:
- APPLICATION_TYPE (17 values)
- AFFILIATION ( 6 values)
- CLASSIFICATION ( 71 values)
- USE_CASE ( 5 values )
- ORGANIZATION ( 4 values )
- STATUS ( 2 values )
- INCOME_AMT ( 9 values )
- SPECIAL_CONSIDERATIONS ( values )
- ASK_AMT ( 8747 values )
The target is: 
- IS_SUCCESSFUL  ( 2 values )
- The data set was edited as follows:
1. The columns 'EIN' and 'NAME' were dropped as uninformative.
2. The column 'APPLICATION_TYPES' has 17 unique values, but many of them occur infrequently.  'APPLICATION_TYPES' with less than 500 counts were condensed into a single 'Other' category.
3. The column 'CLASSIFICATION' has 71 unique values, 26 of them singletons. The single count 'CLASSIFICATION' values were consolidated into an 'Other' category
4. OneHot encoding was used to convert categorical values into numeric values
### A keras sequential model was built with these characteristics:
> - input layer: 2 units, activation="relu"
> - 1 hidden layer with 4 units, activation="relu"
> - 1 output layer with 1 unit, activation="sigmoid"
> - Optimzer: 'Adam'
> - Loss function: binary-crossentropy
> - Metrics: accuracy
### After training on scaled data, the model was run for 100 epochs. It achieved:
> Accuracy: 0.7191
> Loss: 0.5763

## Preprocessing

### - Import dependencies

In [275]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

### - Import pandas and read the charity_data.csv
### from the provided cloud URL into dataframe, and inspect
- 'import pandas as pd'
- 'application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")'
- 'application_df.head()'

In [278]:
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### - drop the  ID columns, 'EIN' and 'NAME'; inspect
- 'application_df.drop(columns=['EIN','NAME'], inplace=True)'
- 'application_df.head()'

In [280]:
application_df.drop(columns=['EIN','NAME'], inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### - determine the number of unique values in each column.
- 'unique_values = application_df.nunique()'
- 'print(unique_values)'

In [282]:
unique_values = application_df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


### - get the number of rows and columns
- 'row_col = application_df.shape'
- 'print(f"Number of rows: {row_col[0]}")'
- 'print(f"Number of columns: {row_col[1]}")'

In [287]:
row_col = application_df.shape
print(f"Number of rows: {row_col[0]}")
print(f"Number of columns: {row_col[1]}")

Number of rows: 34299
Number of columns: 10


### - get the data types of dataframe column values
- 'application_df.info()'

In [290]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


### inspect CLASSIFICATION value counts to identify infrequent types;

In [293]:
counts = application_df['CLASSIFICATION'].value_counts()
count_keys = counts.index
#print(f"keys: {count_keys} values: {counts}")

for i in range(len(counts)):
    print(f"CLASSIFICATION: {count_keys[i]}, COUNT {counts[i]}")

CLASSIFICATION: C1000, COUNT 17326
CLASSIFICATION: C2000, COUNT 6074
CLASSIFICATION: C1200, COUNT 4837
CLASSIFICATION: C3000, COUNT 1918
CLASSIFICATION: C2100, COUNT 1883
CLASSIFICATION: C7000, COUNT 777
CLASSIFICATION: C1700, COUNT 287
CLASSIFICATION: C4000, COUNT 194
CLASSIFICATION: C5000, COUNT 116
CLASSIFICATION: C1270, COUNT 114
CLASSIFICATION: C2700, COUNT 104
CLASSIFICATION: C2800, COUNT 95
CLASSIFICATION: C7100, COUNT 75
CLASSIFICATION: C1300, COUNT 58
CLASSIFICATION: C1280, COUNT 50
CLASSIFICATION: C1230, COUNT 36
CLASSIFICATION: C1400, COUNT 34
CLASSIFICATION: C7200, COUNT 32
CLASSIFICATION: C2300, COUNT 32
CLASSIFICATION: C1240, COUNT 30
CLASSIFICATION: C8000, COUNT 20
CLASSIFICATION: C7120, COUNT 18
CLASSIFICATION: C1500, COUNT 16
CLASSIFICATION: C1800, COUNT 15
CLASSIFICATION: C6000, COUNT 15
CLASSIFICATION: C1250, COUNT 14
CLASSIFICATION: C8200, COUNT 11
CLASSIFICATION: C1238, COUNT 10
CLASSIFICATION: C1278, COUNT 10
CLASSIFICATION: C1235, COUNT 9
CLASSIFICATION: C1237, C

/var/folders/2v/kgb9p_hn6ss9mqhg1hbvvgxh0000gn/T/ipykernel_11651/516944095.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"CLASSIFICATION: {count_keys[i]}, COUNT {counts[i]}")


### - collect the singleton CLASSIFICATIONs in a list

In [296]:
classification_to_replace = []
for i in range(len(counts)):
    if counts.iloc[i] == 1:
        classification = count_keys[i]
        classification_to_replace.append(classification)
print(classification_to_replace)

['C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500', 'C2150']


### - consolidate the singleton CLASSIFICATIONs in 'Other'

In [299]:
for cls in classification_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

### - check that consolidation of CLASSIFICATION was successful

In [302]:
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Other       26
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C1720        6
C2400        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: count, dtype: int64

### Success; the singleton CLASIFICATIONs are consolidated into an 'Other' group with 26 instances

### inspect APPLICATION_TYPE value counts to identify infrequent types;

In [306]:
counts = application_df['APPLICATION_TYPE'].value_counts()
count_keys = counts.index
print(f"keys: {count_keys} values: {counts}")

keys: Index(['T3', 'T4', 'T6', 'T5', 'T19', 'T8', 'T7', 'T10', 'T9', 'T13', 'T12',
       'T2', 'T25', 'T14', 'T29', 'T15', 'T17'],
      dtype='object', name='APPLICATION_TYPE') values: APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [308]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### collect application_types with less than 500 counts
### in list application_types_to_replace 
- 'application_types_to_replace = []'
- 'for i in range(len(counts)):<br>
   > if counts.iloc[i] < 500:<br>
    > > application = count_keys[i]<br>
    > > application_types_to_replace.append(application)'<br>
- 'print(application_types_to_replace) \'

In [311]:
application_types_to_replace = []
for i in range(len(counts)):
    if counts.iloc[i] < 500:
        application = count_keys[i]
        application_types_to_replace.append(application)
print(application_types_to_replace)

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


### - replace low counts (< 500) in dataframe with "Other"
- 'for app in application_types_to_replace:'
    > 'application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")'

In [314]:
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

### - Check to make sure replacement was successful: OK

In [317]:
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [319]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### - Get column headers
- 'application_df.columns'

In [322]:
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

### - generate categorical variable list
- 'application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()'
- 'application_cat'

In [325]:
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

### - check the number of unique values in each catgorical column
- 'application_df[application_cat].nunique()'

In [328]:
application_df[application_cat].nunique()

APPLICATION_TYPE           9
AFFILIATION                6
CLASSIFICATION            46
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

### - create One-Hot Encoder Instance
### NOTE using OneHot encoding rather than get_dummies to deal with categorical values
- 'enc = OneHotEncoder(sparse_output=False)'

In [331]:
enc = OneHotEncoder(sparse_output=False)

### - fit and transform One-Hot Encoder using the categortical variables list 'application_cat'
- 'encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))'

In [334]:
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

### - add the encoder variable names to the dataframe
- 'encode_df.columns = enc.get_feature_names_out(application_cat)'
- 'encode_df.head()'

In [337]:
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### - merge One-Hot features and drop the originals
- 'application_df = application_df.merge(encode_df,left_index=True, right_index=True)'
- 'application_df = application_df.drop(application_cat, axis=1)'
- 'application_df.head()'

In [340]:
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat, axis=1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### - inspect dataframe with 'info'
- 'application_df.info()'

In [343]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 84 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   STATUS                        34299 non-null  int64  
 1   ASK_AMT                       34299 non-null  int64  
 2   IS_SUCCESSFUL                 34299 non-null  int64  
 3   APPLICATION_TYPE_Other        34299 non-null  float64
 4   APPLICATION_TYPE_T10          34299 non-null  float64
 5   APPLICATION_TYPE_T19          34299 non-null  float64
 6   APPLICATION_TYPE_T3           34299 non-null  float64
 7   APPLICATION_TYPE_T4           34299 non-null  float64
 8   APPLICATION_TYPE_T5           34299 non-null  float64
 9   APPLICATION_TYPE_T6           34299 non-null  float64
 10  APPLICATION_TYPE_T7           34299 non-null  float64
 11  APPLICATION_TYPE_T8           34299 non-null  float64
 12  AFFILIATION_CompanySponsored  34299 non-null  float64
 13  A

### - split the preprocessed data into features ('X') and targets ('y')
- 'y = application_df['IS_SUCCESSFUL'].values'
- 'X = application_df.drop("IS_SUCCESSFUL", axis=1).values'

In [346]:
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop("IS_SUCCESSFUL", axis=1).values

### - split the preprocessed data into a training and testing dataset
- 'X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)'

In [349]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

### - Create a StandardScaler instances
- 'scaler = StandardScaler()'

In [352]:
scaler = StandardScaler()

### - Fit the StandardScaler
- 'X_scaler = scaler.fit(X_train)'

In [355]:
X_scaler = scaler.fit(X_train)

### - Scale the data
- 'X_train_scaled = X_scaler.transform(X_train)'
- 'X_test_scaled = X_scaler.transform(X_test)'

In [358]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

### - Define the model: deep neural net
- input layer with 2 neurons; relu activation
- hidden layer with 4 neurons; relu activation
- output layer with 1 neuron; sigmoid activation

### specify the model
- 'nn = tf.keras.models.Sequential()'

In [363]:
nn = tf.keras.models.Sequential()

### - specify the input hidden layer
- 'nn.add(tf.keras.layers.Dense(units=2, activation="relu"))'

In [366]:
nn.add(tf.keras.layers.Dense(units=2, activation="relu"))

### - specify first hidden layer
- 'nn.add(tf.keras.layers.Dense(units=4, activation="relu"))'

In [369]:
nn.add(tf.keras.layers.Dense(units=4, activation="relu"))

### - specify output layer
- 'nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))'

In [372]:
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

### - check the structure of the model

In [375]:
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### - compile the model
- 'nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]'

In [378]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

### - train the model
- 'fit_model = nn.fit(X_train_scaled, y_train, epochs = 100)'

In [381]:
fit_model = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 364us/step - accuracy: 0.5902 - loss: 0.6729
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step - accuracy: 0.6790 - loss: 0.5955
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - accuracy: 0.7132 - loss: 0.5843
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step - accuracy: 0.7088 - loss: 0.5838
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step - accuracy: 0.7164 - loss: 0.5770
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step - accuracy: 0.7176 - loss: 0.5756
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step - accuracy: 0.7110 - loss: 0.5812
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step - accuracy: 0.7134 - loss: 0.5780
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step - accuracy: 0.7069 - loss: 0.5835
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step - accuracy: 0.7152 - loss: 0.5762
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step - accuracy: 0.7191 - loss: 0.5743
Epoch 12/100
804/80

### - evaluate the model using the test data
- 'model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)'
- 'print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")'

In [386]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 564us/step - accuracy: 0.7191 - loss: 0.5763
Loss: 0.5763082504272461, Accuracy: 0.7190670371055603


### - export our model to HDF5 file

In [390]:
nn.save('AlphabetSoupCharity.h5')